In [35]:
library(GWENA)
library(WGCNA)
library(magrittr) 
library(tidyverse) 

In [3]:
exprs <- read.delim(snakemake@input$vst, sep="\t", as.is=TRUE, check.names=FALSE, row.names=1)
exprs <- exprs[rowMeans(exprs)>5,]

In [4]:
sample.info <- read.delim("data/tmp/sample_info.tsv", sep="\t", row.names=1)
sample.info <- sample.info[colnames(exprs),]
gene.info <- read.delim("data/tmp/rnaseq/quant/salmon/tximport/gene_info.tsv", sep="\t", row.names=1, as.is=TRUE)
gene.info <- gene.info[rownames(exprs),]

In [5]:
head(exprs)

In [6]:
head(sample.info)

In [106]:
head(gene.info)

In [107]:
se <- SummarizedExperiment::SummarizedExperiment(assays=list(expr=exprs), colData=sample.info, rowData=gene.info)
dim(se)

**Filter expression data**

In [7]:

se_filt <- filter_low_var(t(exprs), pct = 0.5, type = "median")

In [8]:
dim(se_filt)

In [10]:
require(parallel)
require(doParallel)
cpucores <- makeCluster(detectCores(), type='PSOCK') ; 
registerDoParallel(cpucores) ; 

In [19]:
powers = c(c(1:10), seq(from = 12, to = 70, by = 2))
sft = pickSoftThreshold(se_filt, powerVector = powers,verbose = 5, networkType = "signed")

In [20]:
par(mfrow = c(1,2));
cex1 = 0.9;

plot(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     xlab = "Soft Threshold (power)",
     ylab = "Scale Free Topology Model Fit, signed R^2",
     main = paste("Scale independence")
)
text(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     labels = powers, cex = cex1, col = "red"
)
abline(h = 0.90, col = "red")
plot(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     xlab = "Soft Threshold (power)",
     ylab = "Mean Connectivity",
     type = "n",
     main = paste("Mean connectivity")
)
text(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     labels = powers,
     cex = cex1, col = "red")

In [73]:
picked_power = 30
temp_cor <- cor       
cor <- WGCNA::cor 
netwk <- blockwiseModules(se_filt,                # <= input here

                          # == Adjacency Function ==
                          power = picked_power,                # <= power here
                          networkType = "signed",

                          # == Tree and Block Options ==
                          deepSplit = 2,
                          pamRespectsDendro = F,
                          # detectCutHeight = 0.75,
                          minModuleSize = 30,
                          maxBlockSize = 4000,

                          # == Module Adjustments ==
                          reassignThreshold = 0,
                          mergeCutHeight = 0.5,

                          # == TOM == Archive the run results in TOM file (saves time)
                          saveTOMs = T,
                          saveTOMFileBase = "ER",

                          # == Output Options
                          numericLabels = T,
                          verbose = 3)

In [74]:
mergedColors = labels2colors(netwk$colors)
plotDendroAndColors(
  netwk$dendrograms[[1]],
  mergedColors[netwk$blockGenes[[1]]],
  "Module colors",
  dendroLabels = FALSE,
  hang = 0.03,
  addGuide = TRUE,
  guideHang = 0.05 )


In [75]:
MEs <- moduleEigengenes(se_filt, mergedColors)$eigengenes
MEs <- orderMEs(MEs)
module_order = names(MEs) %>% gsub("ME","", .)
MEs$treatment = sample.info$Sample_Group

In [76]:
# tidy & plot data
mME = MEs %>%
  pivot_longer(-treatment) %>%
  mutate(
    name = gsub("ME", "", name),
    name = factor(name, levels = module_order)
  )

mME %>% ggplot(., aes(x=treatment, y=name, fill=value)) +
  geom_tile() +
  theme_bw() +
  scale_fill_gradient2(
    low = "blue",
    high = "red",
    mid = "white",
    midpoint = 0,
    limit = c(-1,1)) +
  theme(axis.text.x = element_text(angle=90)) +
  labs(title = "Module-trait Relationships", y = "Modules", fill="corr")


In [77]:
MEs <- moduleEigengenes(se_filt, mergedColors)$eigengenes
MEDiss = 1-cor(MEs)
METree = hclust(as.dist(MEDiss), method = "average")
#Plotting the result
#sizeGrWindow(7, 6)
plot(METree, main = "Clustering of module eigengenes",
     xlab = "", sub = "")
MEDissThres = 0.5
#Plotting a cut-off line
abline(h=MEDissThres, col = "red")

In [79]:
#Grouping module colors
#mergedColors = merge$colors
geneTree = netwk$dendrograms[[1]]; 
#Eigengenes of new grouped modules
#mergedMEs = merge$newMEs
#getwd()
#sizeGrWindow(12, 9)
#pdf(file = "/media/natalia/3C8E068E2A640DD5/WGCNA/Spodoptera/geneDendro-3.pdf", wi = 9, he = 6)
plotDendroAndColors(geneTree, cbind(dynamicColors, mergedColors),
                    c("Dynamic Tree Cut", "Merged dynamic"),
                    dendroLabels = FALSE, hang = 0.03,
                    addGuide = TRUE, guideHang = 0.05)

In [80]:
datTraits <- model.matrix(~0 + Sample_Group, data=sample.info)
colnames(datTraits) <- levels(factor(sample.info$Sample_Group))
moduleTraitCor = cor(MEs, datTraits, use = "p")
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, ncol(sample.info))

In [81]:
# Will display correlations and their p-values
textMatrix =  paste(signif(moduleTraitCor, 2), "\n(",
                           signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
               xLabels = colnames(datTraits),
               yLabels = names(MEs),
               ySymbols = names(MEs),
               colorLabels = FALSE,
               colors = greenWhiteRed(50),
               textMatrix = textMatrix,
               setStdMargins = FALSE,
               cex.text = 0.5,
               zlim = c(-1,1),
               main = paste("Module-trait relationships"))


In [71]:
names(datTraits)

In [89]:
ss <- sample.info[,c("Sample_Group"), drop=FALSE]
#ss$Sample_Group <- factor(ss$Sample_Group)
association <- associate_phenotype(MEs, ss)

In [87]:
phenotype_mat <- data.frame(phenA = sample(c("X", "Y", "Z"), 20,
                            replace = TRUE),
                            phenB = sample(c("U", "V"), 20,
                            replace = TRUE),
                            stringsAsFactors = FALSE)

In [91]:
dim(MEs)

In [92]:
eigengene_mat <- data.frame(mod1 = rnorm(20, 0.1, 0.2),
mod2 = rnorm(20, 0.2, 0.2))
phenotype_mat <- data.frame(phenA = sample(c("X", "Y", "Z"), 20,
                            replace = TRUE),
                            phenB = sample(c("U", "V"), 20,
                            replace = TRUE),
                            stringsAsFactors = FALSE)
association <- associate_phenotype(eigengene_mat, phenotype_mat)

In [94]:
dim(eigengene_mat)

In [95]:
dim(phenotype_mat)